# Daha Karmaşık Yapıların Temel Bileşeni: Disiplin

## Kaldığımız Yer

Önceki safhada biraz yol katettik fakat bu teknikle devam edersek belki çöp yerine isim üretebilen bir "çöp" üreteceğiz. En nihayetinde yapmaya çalıştığımız şey fonksiyon üreten bir fonksiyon; bunu inşa etmek için yaptığımız işlemleri soyutlayıp bir disiplin çerçevesinde düzenleyemezsek sisteme dahil edeceğimiz karmaşıklığı yönetemeyeceğiz. Bunun için neler yaptık bir bakalım:

- Lineer denklemleri "nöron" olarak kullandık.
- Aktivasyon için hiperbolik tanjant kullandık.
- Harfleri 2 boyutta ifade etmek için `embedding` kullandık.
- Bunları sıralı olarak işleme aldık.
- Bu karmaşıklığı yönetemediğimiz için "yığın normalleştirme" işine bakmadık.

Bu işlemleri ilk biz yapmadık ve bu karmaşıklık problemi ile ilk kez biz karşılaşmıyoruz; dolayısıyla bu yoldan bizden önce geçen fanilerin çıktılarından faydalanabiliriz. Mesela, [`torch.nn`](https://pytorch.org/docs/stable/nn.html) kütüphanesindeki bazı nesneleri (ki bu nesnelerde bu alanda yapılmış araştırmalardaki yöntemlerin soyut halleri) elle oluşturup onları kullanarak gerçekten çok katmanlı bir algılayıcı yapabiliriz.

Ama önce yine bir [makale](https://arxiv.org/abs/1609.03499)! Konu farklı olsa da katmanlar arası iletişimin daha sağlıklı olması adına buradaki hiyerarşi kavramını kullanacağız.

Aman dikkat! Ne kadar nöron, katman, hiyerarşi gibi soyutlamaları kullanıyor olsak da günün sonunda elimizde (bizim mütevazi boyutlarımızda bile 70 bin civarı parametresi olan) devasa bir fonksiyon var. Bu parametreleri "eğitmek" istiyoruz: elimizde çok daha devasa X-Y çiftleri var; parametreleri öyle düzenlemeliyiz ki `f(x) = y + z` olsun ve z olabildiğince küçük olsun. Ama aynı zamanda fonksiyon "öğrensin", "ezberlemesin" istiyoruz: eğitim setinde olmayan bir x için makul bir y üretebilsin. Bunu "bütün" haliyle yönetemediğimiz için kavramlara bölerek yönetmeye çalışıyoruz. Bu parçaları doğrudan birbirine bağladığımızda "katman" dediğimiz kavram anlamını yitiriyor. Bunun önüne geçmek için `tanh` gibi doğrusal akışı bozan ajanlar ekliyoruz. Bu gibi çözümler eğitim esnasında bazı nöronların "kör" olmasına neden oluyor. Bunu çözmek için (tek seferde tümünü kullanamayacağımız kadar çok olan) eğitim setinden oluşturduğumuz küçük yığınları normalize etmeye çalışıyoruz. Katman kavramı tekrar anlamlı oluyor fakat bu sefer katmanlar arası çok hızlı bilgi geçirdiğimiz için eğitim "yavan" oluyor. İşte bunu çözmek üzere arkadaşlar "nöronlar" önce bir kendi aralarında grup çalışmasıyla eğitimi pekiştirsinler, sonra "üst katmana" sonuçları arz etsinler gibi bir öneri getirmiş. Biz bunu `torch.nn` kütüphanesindeki `flatten` gibi uygulayacağız; arada ufak bir nüans olduğu için bizimki `FlattenConsecutive` olacak. Bahsettiğimiz normalizasyon işini ise `BatchNorm1d` yapacak.

## Hazırlık

In [1]:
# "hiper" parametreler:
n_blok = 8
n_katman = 128
n_embed = 16
n_girdi = n_blok * n_embed
n_batch = 32

In [2]:
# gerekli kütüphaneler
import torch
import torch.nn.functional as F
import random

# bunlar bize hep lazım
alfabe = list('.abcçdefgğhıijklmnoöprsştuüvyz')
harf2idx = { harf:idx for idx, harf in enumerate(alfabe) }
idx2harf = { idx:harf for harf, idx in harf2idx.items() }

# artık bigram ile işimiz yok. bize x y lazım ve buradaki x blok uzunluğunda harf dizisi, y ise bir sonraki harf
def xyOlustur(isimler):
  X, Y = [], []
  for isim in isimler:
    baglam = [0] * n_blok
    for idx in map(lambda harf: harf2idx[harf], isim):
      X.append(baglam)
      Y.append(idx)
      baglam = baglam[1:] + [idx]
  X, Y = torch.tensor(X), torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

# isimleri okurken işleyelim
isimler = list(map(lambda isim: list(isim) + ['.'], open("./isimler.txt", "r").read().splitlines()))

# adetlere devam:
n_alfabe = len(alfabe)
n_isim = len(isimler)
n_isim, isimler[0]

(29996, ['a', 'b', 'a', 'c', 'a', '.'])

## Veri Kümeleri

In [3]:
tumX, tumY = xyOlustur(isimler)
random.seed(5)
trnX, trnY = xyOlustur(random.sample(isimler, k = int(0.8 * n_isim)))

torch.Size([282166, 8]) torch.Size([282166])
torch.Size([225789, 8]) torch.Size([225789])


## Soyutlamalar

In [4]:
# -----------------------------------------------------------------------------------------------
class Sequential:

  def __init__(self, layers):
    self.layers = layers

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    self.out = x
    return self.out

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

  def changeMode(self, training):
    for layer in self.layers:
      layer.training = training

  @torch.no_grad()
  def getReady(self, weight = None):
    n_param = 0
    for l in self.layers:
      l.training = True
      for p in l.parameters():
        p.requires_grad = True
        n_param += p.nelement()
      if weight is not None and hasattr(l, 'weight'):
        l.weight *= weight
    return n_param

  def train(self, X, Y, lr):
    backToEval = not self.layers[0].training
    self.changeMode(True)
    loss = F.cross_entropy(self(X), Y)
    loss.backward()
    for p in self.parameters():
      p.data += -lr * p.grad
      p.grad = None
    if backToEval:
      self.changeMode(False)
    return loss

  @torch.no_grad()
  def evalLoss(self, X, Y):
    backToTraining = self.layers[0].training
    self.changeMode(False)
    loss = F.cross_entropy(self(X), Y)
    if backToTraining:
      self.changeMode(True)
    return loss

# -----------------------------------------------------------------------------------------------
class Linear:

  def __init__(self, fan_in, fan_out, bias = False):
    self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 # kaiming init
    self.bias = torch.zeros(fan_out) if bias else None

  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out

  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

# -----------------------------------------------------------------------------------------------
class Tanh:

  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

# -----------------------------------------------------------------------------------------------
class Embedding:

  def __init__(self, num_embeddings, embedding_dim):
    self.weight = torch.randn((num_embeddings, embedding_dim))

  def __call__(self, IX):
    self.out = self.weight[IX]
    return self.out

  def parameters(self):
    return [self.weight]

# -----------------------------------------------------------------------------------------------
class BatchNorm1d:

  def __init__(self, dim, eps = 1e-5, momentum = 0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      if x.ndim == 2:
        dim = 0
      elif x.ndim == 3:
        dim = (0,1)
      xmean = x.mean(dim, keepdim=True) # batch mean
      xvar = x.var(dim, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

# -----------------------------------------------------------------------------------------------
class FlattenConsecutive:

  def __init__(self, n):
    self.n = n

  def __call__(self, x):
    B, T, C = x.shape # batch, time, channels
    x = x.view(B, T // self.n, C * self.n)
    if x.shape[1] == 1:
      x = x.squeeze(1)
    self.out = x
    return self.out

  def parameters(self):
    return []

## Model ve Parametreler

In [5]:
torch.manual_seed(5)

model = Sequential([
  Embedding(n_alfabe, n_embed),
  FlattenConsecutive(2), Linear(n_embed  * 2, n_katman), BatchNorm1d(n_katman), Tanh(),
  FlattenConsecutive(2), Linear(n_katman * 2, n_katman), BatchNorm1d(n_katman), Tanh(),
  FlattenConsecutive(2), Linear(n_katman * 2, n_katman), BatchNorm1d(n_katman), Tanh(),
  Linear(n_katman, n_alfabe),
])

print(model.getReady(0.1)) # kaiming init: ağırlıkları azaltılarak başlatalım ki loss uçuk bir yerden başlamasın
model.evalLoss(tumX, tumY).item()

74720


3.4011964797973633

## Eğitim

In [6]:
torch.manual_seed(5)

for i in range(9000):

  bat = torch.randint(0, trnX.shape[0], (n_batch,))
  batX, batY = trnX[bat], trnY[bat]

  lr = 0.1 if i < 7500 else 0.01
  loss = model.train(batX, batY, lr)

  if i % 1000 == 0: 
    print('trn', loss.item(), 'tum', model.evalLoss(tumX, tumY).item())

trn 3.4081709384918213 tum 3.4011030197143555
trn 2.362731456756592 tum 2.3986237049102783
trn 2.6051926612854004 tum 2.3217108249664307
trn 2.3475451469421387 tum 2.23840069770813
trn 2.3913252353668213 tum 2.1754918098449707
trn 2.2034337520599365 tum 2.1586201190948486
trn 2.04197096824646 tum 2.083258628845215
trn 2.235935688018799 tum 2.071554660797119
trn 1.894763708114624 tum 1.983964443206787


## Sonuç

In [7]:
torch.manual_seed(5)
model.changeMode(training = False)
for _ in range(25):
  ornek, baglam = [0], [0] * n_blok
  while True:
    P = F.softmax(model(torch.tensor([baglam])), dim = 1)
    idx = torch.multinomial(P, num_samples = 1).item()
    baglam = baglam[1:] + [idx]
    ornek.append(idx)
    if idx == 0:
      break
  print(''.join(idx2harf[i] for i in ornek))

.karaç.
.çomurlu.
.karefentli.
.megoğlu.
.teseemin.
.kızkılı.
.hacıançay.
.gücekçazıkçık.
.beymercek.
.umekli.
.ıcıçıcat.
.binat.
.cemiti.
.koyapınar.
.bolanpınar.
.akçılugöme.
.kopralan.
.hatunt.
.asin.
.bukçatoğun.
.hacıamanlı.
.setiran.
.murdere.
.gılıhpınar.
.sesşinköy.
